# DSCI351 Project: ChatDB

# 1️⃣ Set Up The Environment

In [1]:
#Set up the environment

import pymysql
import pymongo
import sqlalchemy
import pandas as pd
import openai
import ipywidgets
print("✅All Resource Installed Successfully！")


✅All Resource Installed Successfully！


# 2️⃣ RDBMS Part: MySQL

### 2.1 Preparation Work (Connect to mysql service, create database, import data, etc.)

In [5]:
# Connect to mysql service
import pymysql

conn = pymysql.connect(
    host="",
    user="",
    password=""
) # Replace AWS address infomation

with conn.cursor() as cursor:
    cursor.execute("CREATE DATABASE IF NOT EXISTS product_db;")
    cursor.execute("CREATE DATABASE IF NOT EXISTS customer_db;")
    cursor.execute("CREATE DATABASE IF NOT EXISTS order_db;")
    print("✅ DataBase create Successfully")

conn.commit()
conn.close()


✅ DataBase create Successfully


In [7]:
#Import Data to product_db
import pandas as pd
from sqlalchemy import create_engine

# Connect to product_db
engine_product = create_engine(" ")# Replace AWS address infomation

product_tables = {
    "products.csv": "products",
    "categories.csv": "categories"
}

# Import Data
for file, table in product_tables.items():
    df = pd.read_csv(file)
    df.to_sql(table, con=engine_product, if_exists="replace", index=False)
    print(f"✅ Imported {file} to product_db.{table}")

✅ Imported products.csv to product_db.products
✅ Imported categories.csv to product_db.categories


In [8]:
#Import Data to customer_db

# connect to customer_db
engine_customer = create_engine(" ")# Replace AWS address infomation

customer_tables = {
    "customers.csv": "customers",
    "cities.csv": "cities",
    "countries.csv": "countries"
}

# Import Data
for file, table in customer_tables.items():
    df = pd.read_csv(file)
    df.to_sql(table, con=engine_customer, if_exists="replace", index=False)
    print(f"✅ Imported {file} to customer_db.{table}")

✅ Imported customers.csv to customer_db.customers
✅ Imported cities.csv to customer_db.cities
✅ Imported countries.csv to customer_db.countries


In [10]:
#Import Data to order_db
import pandas as pd
from sqlalchemy import create_engine

# Connect to product_db
engine_order = create_engine("")# Replace AWS address infomation

order_tables = {
    "orders.csv": "orders",
    "order_customers.csv": "order_customers"
}

# Import Data
for file, table in order_tables.items():
    df = pd.read_csv(file)
    df.to_sql(table, con=engine_product, if_exists="replace", index=False)
    print(f"✅ Imported {file} to order_db.{table}")

✅ Imported orders.csv to order_db.orders
✅ Imported order_customers.csv to order_db.order_customers


### 2.2 OpenAI LLM Prompt

In [14]:
sql_prompt_product = """
You are a SQL expert. Given a user's natural language request, generate the appropriate SQL query. 
You can use any valid SQL operation, including SELECT, INSERT, UPDATE, DELETE, CREATE, etc., depending on what the user is asking for.
Return only the SQL code without any explanation.

The database schema includes the following tables in database `product_db`:

Table: products (ProductID, ProductName, Price, CategoryID)
Table: categories (CategoryID, CategoryName)

❗Rules:
- Do NOT include explanations, only return the SQL SELECT statement.
- Use JOINs if necessary.
- Assume the database is named product_db.

Note: Whenever use GROUP BY, remember that all non-aggregate fields must appear in GROUP BY (MySQL default behavior) or use aggregate functions on these fields (such as MIN(cities.CityName))
Note: Use OFFSET for skip, and the format should be "LIMIT a OFFSET b"

For schema exploration, always use: SHOW TABLES;
Do not use `information_schema.tables` unless explicitly asked for schema metadata.
"""

sql_prompt_customer = """
You are a SQL expert. Given a user's natural language request, generate the appropriate SQL query. 
You can use any valid SQL operation, including SELECT, INSERT, UPDATE, DELETE, CREATE, etc., depending on what the user is asking for.
Return only the SQL code without any explanation.

The database schema includes the following tables in database `customer_db`:

Table: customers (CustomerID, FirstName, MiddleInitial, LastName, CityID, Address)
Table: cities (CityID, CityName, Zipcode, CountryID)
Table: countries (CountryID, CountryName, CountryCode)

Note that there is no CountryID in table customers, so have to connect customers with cities and countries to check for countries.

❗Rules:
- Do NOT include explanations, only return the SQL SELECT statement.
- Use JOINs if necessary.
- Assume the database is named customer_db.

Note: Whenever use GROUP BY, remember that all non-aggregate fields must appear in GROUP BY (MySQL default behavior) or use aggregate functions on these fields (such as MIN(cities.CityName))
Note: Use OFFSET for skip, and the format should be "LIMIT a OFFSET b"

For schema exploration, always use: SHOW TABLES;
Do not use `information_schema.tables` unless explicitly asked for schema metadata.
"""


sql_prompt_order = """
You are a SQL expert. Given a user's natural language request, generate the appropriate SQL query. 
You can use any valid SQL operation, including SELECT, INSERT, UPDATE, DELETE, CREATE, etc., depending on what the user is asking for.
Return only the SQL code without any explanation.

The database schema includes the following tables in database `order_db`:

Table: orders (order_id, order_date, customer_id, total_price) 
Table: order_customers (customer_id, first_name, last_name, address, email, phone_number)

Note: If all the infomation that given and request are in the same table, do NOT need to use join.
Note: Whenever use GROUP BY, remember that all non-aggregate fields must appear in GROUP BY (MySQL default behavior) or use aggregate functions on these fields (such as MIN(cities.CityName))
Note: Use OFFSET for skip, and the format should be "LIMIT a OFFSET b"

❗Rules:
- Do NOT include explanations, only return the SQL SELECT statement.
- Use JOINs if necessary.
- Assume the database is named order_db.

For schema exploration, always use: SHOW TABLES;
Do not use `information_schema.tables` unless explicitly asked for schema metadata.
"""

### 2.3 Select the Data Base

In [17]:
selected_db = "customer"  # Choose from "customer" or "product" or "order"

In [19]:
# Choose the database
import openai

client = openai.OpenAI(api_key="")  # Replace OpenAI API key

if selected_db == "product":
    sql_prompt = sql_prompt_product
    engine = engine_product
elif selected_db == "customer":
    sql_prompt = sql_prompt_customer
    engine = engine_customer
elif selected_db == "order":
    sql_prompt = sql_prompt_order
    engine = engine_order

### 2.4 Input the Natural Language

In [30]:
user_input_test = "What is the address of customer named Lee Zhang" #User input the natural language query

### 2.5 LLM Convert Natural Language to SQL query Language

In [33]:
#Receive Request and convert to SQL Language

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": sql_prompt},
        {"role": "user", "content": user_input_test}
    ]
)

generated_sql = response.choices[0].message.content.strip()

if generated_sql.startswith("```sql"):
    generated_sql = generated_sql[6:]  
if generated_sql.endswith("```"):
    generated_sql = generated_sql[:-3]  

print("🧠 GPT generated SQL query：\n", generated_sql)


🧠 GPT generated SQL query：
 SELECT Address
FROM customers
WHERE FirstName = 'Lee' AND LastName = 'Zhang';


### 2.6 Output the Query Result

In [36]:
from sqlalchemy import text

sql_statements = [stmt.strip() for stmt in generated_sql.split(";") if stmt.strip()]

for stmt in sql_statements:
    try:
        with engine.connect() as conn:
            result = conn.execute(text(stmt))

            if stmt.strip().lower().startswith(("select", "show", "describe")):
                rows = result.fetchall()
                print(f"\n📄 Query Result is")
                for row in rows:
                    print(row)
            else:
                conn.commit()
                print(f"\n✅ Data operation executed successfully for: {stmt}")
    except Exception as e:
        print(f"\n❌ Failed to execute: {stmt}")
        print("   Error:", e)


📄 Query Result is
('921 White Fabien Avenue',)


# 3️⃣ NoSQL Part: MongoDB

## DataBase#1: Customer

### Prepare Work

In [40]:
from pymongo import MongoClient
import pandas as pd
import openai

client = MongoClient("")#Replace AWS address infomation
db = client["customer_db"]

files_and_collections = {
    "countries.csv": "countries",
    "cities.csv": "cities",
    "customers.csv": "customers"
}

for file, collection_name in files_and_collections.items():
    df = pd.read_csv(file)
    data = df.to_dict(orient="records")
    collection = db[collection_name]
    collection.insert_many(data)
    print(f"✅ Imported {file} to '{collection_name}' collection successfully!")

client_ai = openai.OpenAI(api_key="")#Replace OpenAI API key


✅ Imported countries.csv to 'countries' collection successfully!
✅ Imported cities.csv to 'cities' collection successfully!
✅ Imported customers.csv to 'customers' collection successfully!


### Prompt

In [42]:
def generate_prompt(user_query):
    return f"""
You are a MongoDB assistant.

You are working with a MongoDB database. The main variable is db. It has three collections:
- countries (CountryID, CountryName, CountryCode)
- cities (CityID, CityName, Zipcode, CountryID)
- customers (CustomerID, FirstName, MiddleInitial, LastName, CityID, Address)

Understand the user's natural language query and return a **valid MongoDB command using PyMongo** in Python.

⚠️ Always do the following:
- Assign the result to a variable named `results`, e.g., `results = db.customers.find({...})`
- If it is an insert, assign to `result`, e.g., `result = db.countries.insert_one({...})`
- If it is a delete, assign to `result`, e.g., `result = db.cities.delete_many({...})`
- If it is an aggregation, assign to `results`, e.g., `results = db.customers.aggregate([...])`
- If it is an update, assign to `result`, e.g., `result = db.customers.update_one({{...}}, {{'$set': {{...}}}})`
    - If the update value is fetched from another collection (e.g., setting `CityID` to the ID of a city), use `find_one()` first and assign to a temp variable, then use it in `update_one`
- If listing collection names, assign to `results`, e.g., `results = db.list_collection_names()`

When using $lookup to join collections:
- Join customers with cities using CityID
- Join cities with countries using CountryID
- You may use nested $lookup if needed
- If $lookup is used, always follow with $unwind before $match.

Always use $project to return only relevant fields.
Always return a single variable named `results` or `result` depending on context.
Return only executable Python code (no markdown, no explanation).

User query: "{user_query}"
"""


### User Input

In [46]:
user_query = "How many people are from each city, only show cities with people less than 1000"

### OpenAI Generate Query

In [49]:
prompt = generate_prompt(user_query)

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

mongo_query_str = response.choices[0].message.content
print("ChatGPT generated MongoDB query:\n")
print(mongo_query_str)


ChatGPT generated MongoDB query:

results = db.customers.aggregate([
    {
        "$lookup": {
            "from": "cities",
            "localField": "CityID",
            "foreignField": "CityID",
            "as": "city_info"
        }
    },
    {
        "$unwind": "$city_info"
    },
    {
        "$group": {
            "_id": "$city_info.CityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$lt": 1000}
        }
    },
    {
        "$project": {
            "_id": 0,
            "CityName": "$_id",
            "Count": "$count"
        }
    }
])


### Final Output

In [52]:
import json

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": generate_prompt(user_query)}],
    temperature=0
)

code_str = response.choices[0].message.content.strip()

if code_str.startswith("```python"):
    code_str = code_str[len("```python"):].strip()
if code_str.endswith("```"):
    code_str = code_str[:-3].strip()

try:
    exec_locals = {"db": db}
    exec(code_str, {}, exec_locals)

    # ✅ Query result only when 'results' is actually defined
    if "results" in exec_locals:
        print("\n📋 Query Result:\n")
        results = exec_locals["results"]

        if isinstance(results, list):
            for doc in results:
                print(doc)
        elif isinstance(results, dict):
            print(results)
        elif hasattr(results, "__iter__") and not isinstance(results, str):
            print("📎 Cursor result：\n")
            for doc in list(results):
                print(doc)
        else:
            print(results)

    # Insert
    if "insert_one" in code_str or "insert_many" in code_str:
        print("✅ Insert Successfully:", exec_locals.get("result", "OK"))

    # Delete
    if "delete_one" in code_str or "delete_many" in code_str:
        print("🗑 Delete Successfully，Number:", exec_locals["result"].deleted_count)

    # Update
    if "update_one" in code_str or "update_many" in code_str:
        print("🔧 Update Successfully，Match:", exec_locals["result"].matched_count)
        print("📝 Change:", exec_locals["result"].modified_count)

    print("\n✅ Finished")

except Exception as e:
    print("❌ Error:", e)

KeyboardInterrupt: 

## DataBase#2: Products

### Prepare Work

In [56]:
from pymongo import MongoClient
import pandas as pd
import openai

client = MongoClient("")#Replace AWS address infomation
db = client["products_db"]

files_and_collections = {
    "categories.csv": "categories",
    "products.csv": "products"
}

for file, collection_name in files_and_collections.items():
    df = pd.read_csv(file)
    data = df.to_dict(orient="records")
    collection = db[collection_name]
    collection.insert_many(data)
    print(f"✅ Imported {file} to '{collection_name}' collection successfully!")

client_ai = openai.OpenAI(api_key="")#Replace OpenAI API key

✅ Imported categories.csv to 'categories' collection successfully!
✅ Imported products.csv to 'products' collection successfully!


### Prmopt

In [59]:
def generate_prompt(user_query):
    return f"""
You are a MongoDB assistant.

You are working with a MongoDB database. The main variable is db. It has two collections:
- products (ProductID, ProductName, Price, CategoryID, Class, ModifyDate, Resistant, IsAllergic, VitalityDays)
- categories (CategoryID, CategoryName)

Understand the user's natural language query and return a **valid MongoDB command using PyMongo** in Python.

⚠️ Always do the following:
- Assign the result to a variable named `results`, e.g., `results = db.customers.find({...})`
- If it is an insert, assign to `result`, e.g., `result = db.countries.insert_one({...})`
- If it is a delete, assign to `result`, e.g., `result = db.cities.delete_many({...})`
- If it is an aggregation, assign to `results`, e.g., `results = db.customers.aggregate([...])`
- If it is an update, assign to `result`, e.g., `result = db.customers.update_one({{...}}, {{'$set': {{...}}}})`
    - If the update value is fetched from another collection (e.g., setting `CityID` to the ID of a city), use `find_one()` first and assign to a temp variable, then use it in `update_one`
- If listing collection names, assign to `results`, e.g., `results = db.list_collection_names()`

When using $lookup to join collections:
- Join customers with cities using CityID
- Join cities with countries using CountryID
- You may use nested $lookup if needed
- If $lookup is used, always follow with $unwind before $match.

Always use $project to return only relevant fields.
Always return a single variable named `results` or `result` depending on context.
Return only executable Python code (no markdown, no explanation).

User query: "{user_query}"
"""

### User Input

In [70]:
user_query = "calculate the average price of each category, keep only categories with average price less than 50, and sort by average price in descending order."

### OpenAI Generate Query

In [73]:
prompt = generate_prompt(user_query)

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

mongo_query_str = response.choices[0].message.content
print("ChatGPT generated MongoDB query:\n")
print(mongo_query_str)


ChatGPT generated MongoDB query:

results = db.products.aggregate([
    {
        "$lookup": {
            "from": "categories",
            "localField": "CategoryID",
            "foreignField": "CategoryID",
            "as": "category_info"
        }
    },
    {
        "$unwind": "$category_info"
    },
    {
        "$group": {
            "_id": "$category_info.CategoryName",
            "average_price": {"$avg": "$Price"}
        }
    },
    {
        "$match": {
            "average_price": {"$lt": 50}
        }
    },
    {
        "$sort": {
            "average_price": -1
        }
    },
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "average_price": 1
        }
    }
])


### Final Output

In [75]:
import json

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": generate_prompt(user_query)}],
    temperature=0
)

code_str = response.choices[0].message.content.strip()

if code_str.startswith("```python"):
    code_str = code_str[len("```python"):].strip()
if code_str.endswith("```"):
    code_str = code_str[:-3].strip()

try:
    exec_locals = {"db": db}
    exec(code_str, {}, exec_locals)

    # ✅ Query result only when 'results' is actually defined
    if "results" in exec_locals:
        print("\n📋 Query Result:\n")
        results = exec_locals["results"]

        if isinstance(results, list):
            for doc in results:
                print(doc)
        elif isinstance(results, dict):
            print(results)
        elif hasattr(results, "__iter__") and not isinstance(results, str):
            print("📎 Cursor result：\n")
            for doc in list(results):
                print(doc)
        else:
            print(results)

    # Insert
    if "insert_one" in code_str or "insert_many" in code_str:
        print("✅ Insert Successfully:", exec_locals.get("result", "OK"))

    # Delete
    if "delete_one" in code_str or "delete_many" in code_str:
        print("🗑 Delete Successfully，Number:", exec_locals["result"].deleted_count)

    # Update
    if "update_one" in code_str or "update_many" in code_str:
        print("🔧 Update Successfully，Match:", exec_locals["result"].matched_count)
        print("📝 Change:", exec_locals["result"].modified_count)

    print("\n✅ Finished")

except Exception as e:
    print("❌ Error:", e)


📋 Query Result:

📎 Cursor result：

{'average_price': 49.460882978723404, 'category': 'Poultry'}
{'average_price': 48.66665555555556, 'category': 'Seafood'}
{'average_price': 45.396100000000004, 'category': 'Produce'}
{'average_price': 44.27278333333334, 'category': 'Shell fish'}

✅ Finished


## DataBase#3: Orders

### Prepare Work

In [520]:
from pymongo import MongoClient
import pandas as pd
import openai

client = MongoClient("")#Replace AWS address infomation
db = client["order_db"]

files_and_collections = {
    "orders.csv": "orders",
    "order_customers.csv": "order_customers"
}

for file, collection_name in files_and_collections.items():
    df = pd.read_csv(file)

    data = df.to_dict(orient="records")
    collection = db[collection_name]
    collection.insert_many(data)
    print(f"✅ Imported {file} into '{collection_name}' collection successfully!")

client_ai = openai.OpenAI(api_key="")#Replace OpenAI API key

✅ Imported orders.csv into 'orders' collection successfully!
✅ Imported order_customers.csv into 'order_customers' collection successfully!


### Prmopt

In [522]:
def generate_prompt(user_query):
    return f"""
You are a MongoDB assistant.

You are working with a MongoDB database. The main variable is db. It has two collections:
- orders (order_id, order_date, customer_id, total_price)
- order_customers (customer_id, first_name, last_name, address, email, phone_number)

Understand the user's natural language query and return a **valid MongoDB command using PyMongo** in Python.

⚠️ Always do the following:
- Assign the result to a variable named `results`, e.g., `results = db.customers.find({...})`
- If it is an insert, assign to `result`, e.g., `result = db.countries.insert_one({...})`
- If it is a delete, assign to `result`, e.g., `result = db.cities.delete_many({...})`
- If it is an aggregation, assign to `results`, e.g., `results = db.customers.aggregate([...])`
- If it is an update, assign to `result`, e.g., `result = db.customers.update_one({{...}}, {{'$set': {{...}}}})`
    - If the update value is fetched from another collection (e.g., setting `CityID` to the ID of a city), use `find_one()` first and assign to a temp variable, then use it in `update_one`
- If listing collection names, assign to `results`, e.g., `results = db.list_collection_names()`

When using $lookup to join collections:
- Join customers with cities using CityID
- Join cities with countries using CountryID
- You may use nested $lookup if needed
- If $lookup is used, always follow with $unwind before $match.

Always use $project to return only relevant fields.
Always return a single variable named `results` or `result` depending on context.
Return only executable Python code (no markdown, no explanation).

Note: for any query including date, just use formate like "2001-01-01" as a string to query, no need to convert anything

User query: "{user_query}"
only generate mongodb query without any other words
"""

### User Input

In [526]:
user_query = ""

### OpenAI Query

In [529]:
prompt = generate_prompt(user_query)

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

mongo_query_str = response.choices[0].message.content

if mongo_query_str.startswith("```python"):
    mongo_query_str = mongo_query_str[len("```python"):].strip()
if mongo_query_str.endswith("```"):
    mongo_query_str = mongo_query_str[:-3].strip()

print("ChatGPT generated MongoDB query:\n")
print(mongo_query_str)


ChatGPT generated MongoDB query:

results = db.orders.aggregate([
    {
        '$lookup': {
            'from': 'order_customers',
            'localField': 'customer_id',
            'foreignField': 'customer_id',
            'as': 'customer_info'
        }
    },
    {
        '$unwind': '$customer_info'
    },
    {
        '$match': {
            'total_price': {'$gt': 900},
            'customer_info.address': '123 Main St, Springfield, IL'
        }
    },
    {
        '$project': {
            '_id': 0,
            'full_name': {'$concat': ['$customer_info.first_name', ' ', '$customer_info.last_name']},
            'price_of_purchase': '$total_price'
        }
    },
    {
        '$sort': {'price_of_purchase': -1}
    },
    {
        '$skip': 2
    },
    {
        '$limit': 5
    }
])


### Final Output

In [532]:
import json

response = client_ai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": generate_prompt(user_query)}],
    temperature=0
)

code_str = response.choices[0].message.content.strip()

if code_str.startswith("```python"):
    code_str = code_str[len("```python"):].strip()
if code_str.endswith("```"):
    code_str = code_str[:-3].strip()

try:
    exec_locals = {"db": db}
    exec(code_str, {}, exec_locals)

    # ✅ Query result only when 'results' is actually defined
    if "results" in exec_locals:
        print("\n📋 Query Result:\n")
        results = exec_locals["results"]

        if isinstance(results, list):
            for doc in results:
                print(doc)
        elif isinstance(results, dict):
            print(results)
        elif hasattr(results, "__iter__") and not isinstance(results, str):
            print("📎 Cursor result：\n")
            for doc in list(results):
                print(doc)
        else:
            print(results)

    # Insert
    if "insert_one" in code_str or "insert_many" in code_str:
        print("✅ Insert Successfully:", exec_locals.get("result", "OK"))

    # Delete
    if "delete_one" in code_str or "delete_many" in code_str:
        print("🗑 Delete Successfully，Number:", exec_locals["result"].deleted_count)

    # Update
    if "update_one" in code_str or "update_many" in code_str:
        print("🔧 Update Successfully，Match:", exec_locals["result"].matched_count)
        print("📝 Change:", exec_locals["result"].modified_count)

    print("\n✅ Finished")

except Exception as e:
    print("❌ Error:", e)

KeyboardInterrupt: 

# Thank You !!! ❤️